In [33]:
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib as plt
import time
from keras.models import load_model


In [34]:
# Perform histogram equalization on the randomly selected image
def equalize(img):
  img = cv2.equalizeHist(img)
  return img

# Convert the randomly selected image to grayscale
def grayscale(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img

# Perform grayscale conversion, histogram equalization and normalization on the whole dataset
def preprocessing(img):
  img = grayscale(img)
  img = equalize(img)
  img = img/255
  return img

In [35]:
best_model = load_model('../CNN_model/cnn_classifier.keras')


In [36]:
# Load weights
path_to_weights = '../yolov4_model/yolov4_tsb_train_best.weights'
# Load configuration file
path_to_cfg = '../yolov4_model/yolov4_tsb_test.cfg'

# Load weights and configuration file into the network
network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)

# To use with GPU
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV) #OPENCV
network.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16) #OPENCL_FP16

In [37]:
# Get names of all YOLO v4 layers
layers_all = network.getLayerNames()

# Get only detection layers
output_layers = network.getUnconnectedOutLayers()
layers_names_output = [layers_all[i - 1] for i in output_layers]


In [38]:
# Load file with the names of traffic signs

labels = pd.read_csv('../data/signnames.csv')
print(labels)

    ClassId                                           SignName
0         0                               Speed limit (20km/h)
1         1                               Speed limit (30km/h)
2         2                               Speed limit (50km/h)
3         3                               Speed limit (60km/h)
4         4                               Speed limit (70km/h)
5         5                               Speed limit (80km/h)
6         6                        End of speed limit (80km/h)
7         7                              Speed limit (100km/h)
8         8                              Speed limit (120km/h)
9         9                                         No passing
10       10       No passing for vechiles over 3.5 metric tons
11       11              Right-of-way at the next intersection
12       12                                      Priority road
13       13                                              Yield
14       14                                            

In [39]:
# Minimum probability to eliminate weak detections

probability_minimum = 0.6

# Set threshold to filter weak bounding boxes by non-maximum suppression
threshold = 0.4

# Generate colours for bounding boxes
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

In [40]:
# Set default size of plots
plt.rcParams['figure.figsize'] = (3, 3)


# Variable for counting total amount of frames
f = 0

# Variable for counting total processing time
t = 0

# Path to the directory containing photos
photos_directory = '../data/img'

# Iterate through photos in the directory
for photo_filename in os.listdir(photos_directory):
    # Construct the full path to the photo
    photo_path = os.path.join(photos_directory, photo_filename)

    # Read the photo
    frame = cv2.imread(photo_path)

    # Get spatial dimensions of the frame for the first time
    h, w = frame.shape[:2]

    # Blob from the current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (800, 800), swapRB=True, crop=False)

    # Forward pass with blob through output layers
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increase counters
    f += 1
    t += end - start

    # Lists for detected bounding boxes, confidences, and class numbers
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Go through all output layers after feed forward pass
    for result in output_from_network:
        # Go through all detections from the current output layer
        for detected_objects in result:
            # Get 80 classes probabilities for the current detected object
            scores = detected_objects[5:]
            # Get the index of the class with the maximum value of probability
            class_current = np.argmax(scores)
            # Get the value of probability for the defined class
            confidence_current = scores[class_current]

            # Eliminate weak predictions by minimum probability
            if confidence_current > probability_minimum:
                # Scale bounding box coordinates to the initial frame size
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                # Get top-left corner coordinates
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                # Add results into prepared lists
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)

    # Implement non-maximum suppression of given bounding boxes
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    # Check if there is any detected object left
    if len(results) > 0:
        # Go through indexes of results
        for i in results.flatten():
            # Bounding box coordinates (width and height)
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            # Cut fragment with a traffic sign
            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]

            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                # Preprocess cut out fragments
                c_ts = np.asarray(c_ts)
                c_ts = cv2.resize(c_ts, (32, 32))
                c_ts = preprocessing(c_ts)
                c_ts = c_ts.reshape(1, 32, 32, 1)

                # Feed to the Classification model to get predicted label among 43 classes
                scores = best_model.predict(c_ts)

                # Get only class with max confidence
                prediction = np.argmax(scores)

                # Color for the current bounding box
                color_box_current = colours[class_numbers[i]].tolist()

                # Draw bounding box on the original current frame
                cv2.rectangle(frame, (x_min, y_min),
                              (x_min + box_width, y_min + box_height),
                              color_box_current, 2)

                                # Prepare text with label and confidence for the current bounding box
                text_box_current = '{}: {:.4f}'.format(labels['SignName'][prediction], confidences[i])

                # Put text with label and confidence on the original image
                cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_box_current, 2)
        # Specify the output filename based on the input photo filename
        output_filename = f"processed_{os.path.splitext(photo_filename)[0]}.jpg"

        output_path = "../results/images"
        # Save the processed frame to the output directory
        cv2.imwrite(os.path.join(output_path, output_filename), frame)

        # Wait for a key event
        cv2.waitKey(0)

# Close all windows after processing all images
cv2.destroyAllWindows()
